# WiDS Datathon 2026 Notebook
This is where your analysis begins. Use this notebook for EDA, modeling, and explanations.

# WiDS Datathon 2026 – Route 1: Accelerating Equitable Evacuations

**Goal:** Identify evacuation alert delays and build an early-trigger recommendation baseline using WatchDuty data.

**Core KPI:** Minutes of lead time gained (how much earlier we could trigger protective action).



In [24]:
# Your code goes here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json          # pentru a interpreta string-uri JSON din coloana "changes"
from collections import Counter  # numărăm frecvențe de chei

# Setări pentru afișare (ca să vedem mai multe coloane)
pd.set_option("display.max_columns", 200)

# Setăm un seed pentru reproducibilitate (același random mereu)
np.random.seed(42)

print("Setup ready ✅")




Setup ready ✅


In [27]:
# Citim fișierul cu evenimente (incendii + locații); noi vom filtra doar wildfire
geo_events = pd.read_csv("/content/geo_events_geoevent.csv", low_memory=False)

# Citim changelog-ul evenimentelor (timeline de modificări + semnale radio)
ge_changelog = pd.read_csv("/content/geo_events_geoeventchangelog.csv", low_memory=False)

# Citim changelog-ul zonelor de evacuare (status changes: warning -> order etc.)
evac_changelog = pd.read_csv("/content/evac_zones_gis_evaczonechangelog.csv", low_memory=False)

# Citim maparea dintre incendii (geo_event_id) și zone (evac_zone_id)
map_event_zone = pd.read_csv("/content/evac_zone_status_geo_event_map.csv", low_memory=False)

# (Opțional) Citim perimetrele focului; nu e obligatoriu în MVP
fire_perimeters = pd.read_csv("fire_perimeters_gis_fireperimeter.csv", low_memory=False)

print("Loaded ✅")
print("geo_events:", geo_events.shape)
print("ge_changelog:", ge_changelog.shape)
print("evac_changelog:", evac_changelog.shape)
print("map_event_zone:", map_event_zone.shape)
print("fire_perimeters:", fire_perimeters.shape)


ParserError: Error tokenizing data. C error: EOF inside string starting at row 6077

In [ ]:
from google.colab import drive
drive.mount('/content/drive')